In [1]:
from sodapy import Socrata
import requests, zipfile, StringIO
import urllib2
import pandas as pd
import numpy as np
import pypyodbc
import csv
import os
import tempfile
import statsmodels.formula.api as smf
import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
puma_ref = pd.read_excel('http://www.nyc.gov/html/dcp/download/census/nyc2010census_tabulation_equiv.xlsx',header=3)
# alternatively, use shapefile from https://data.cityofnewyork.us/Housing-Development/Public-Use-Microdata-Areas-PUMA-/cwiz-gcty
puma_list = puma_ref['PUMA'].unique()[~np.isnan(puma_ref['PUMA'].unique())]

In [3]:
pums_data = pd.read_csv('ss14hny.csv')
pums_data['PUMA'] = pums_data[['PUMA00','PUMA10']].max(axis=1)

In [4]:
nyc_pums = pd.DataFrame()
for puma in puma_list:
    nyc_pums = nyc_pums.append(pums_data.query('PUMA == @puma'))
    print nyc_pums.shape[0],'rows after addition of PUMA',str(puma)

2532 rows after addition of PUMA 3704.0
4290 rows after addition of PUMA 3706.0
7271 rows after addition of PUMA 3705.0
9141 rows after addition of PUMA 3703.0
11081 rows after addition of PUMA 3708.0
13603 rows after addition of PUMA 3702.0
15801 rows after addition of PUMA 3707.0
18926 rows after addition of PUMA 3710.0
21257 rows after addition of PUMA 3701.0
24034 rows after addition of PUMA 3709.0
27707 rows after addition of PUMA 4017.0
30805 rows after addition of PUMA 4013.0
33911 rows after addition of PUMA 4003.0
36770 rows after addition of PUMA 4014.0
38913 rows after addition of PUMA 4018.0
42475 rows after addition of PUMA 4004.0
44772 rows after addition of PUMA 4007.0
46949 rows after addition of PUMA 4002.0
51163 rows after addition of PUMA 4009.0
53726 rows after addition of PUMA 4005.0
56310 rows after addition of PUMA 4006.0
58238 rows after addition of PUMA 4011.0
60981 rows after addition of PUMA 4008.0
63602 rows after addition of PUMA 4010.0
66612 rows after add

In [5]:
AMI_60_max = {1:36300,2:41460,3:46620,4:51780,5:56940,6:62100,7:67260,8:72420}
# 80 pct figures for 2015 from http://www1.nyc.gov/assets/hpd/downloads/pdf/NSP-tenant-lease-guidelines.pdf
AMI_80_max = {1:48400,2:55280,3:62160,4:69040,5:74640,6:80160,7:85680,8:91200}
nyc_pums['AMI_60'] = 0
for n in AMI_60_max.items():
    nyc_pums.loc[(nyc_pums['NP']==n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_60'] = 1
nyc_pums.loc[(nyc_pums['NP']>n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_60'] = 1
nyc_pums['AMI_80'] = 0
for n in AMI_80_max.items():
    nyc_pums.loc[(nyc_pums['NP']==n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_80'] = 1
nyc_pums.loc[(nyc_pums['NP']>n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_80'] = 1

In [6]:
age_grouping = {1:'PreWar',2:'PostWar',3:'PostWar',4:'PostWar',5:'PostWar',6:'PostWar',7:'PostWar',8:'Modern'}
nyc_pums['AgeGroup'] = ''
for n in age_grouping.items():
    nyc_pums.loc[nyc_pums['YBL']==n[0],'AgeGroup'] = n[1]
nyc_pums.loc[nyc_pums['YBL']>n[0],'AgeGroup'] = n[1]

In [7]:
# size_grouping = {1:'Small',2:'Small',3:'Small',4:'Small',5:'Medium',6:'Medium',7:'Medium',8:'Large',9:'Large'}
size_grouping = {1:'Small',2:'Small',3:'Small',4:'Small',5:'Small',6:'Large',7:'Large',8:'Large',9:'Large'}
nyc_pums['SizeGroup'] = ''
for n in size_grouping.items():
    nyc_pums.loc[nyc_pums['BLD']==n[0],'SizeGroup'] = n[1]
nyc_pums.loc[nyc_pums['BLD']>n[0],'SizeGroup'] = n[1]

In [8]:
nyc_pums['ResidencyGroup'] = ''
nyc_pums.loc[nyc_pums['MV']<=4,'ResidencyGroup'] = 'Under10'
nyc_pums.loc[nyc_pums['MV']==5,'ResidencyGroup'] = '10Years'
nyc_pums.loc[nyc_pums['MV']==6,'ResidencyGroup'] = '20Years'
nyc_pums.loc[nyc_pums['MV']==7,'ResidencyGroup'] = '30Years'

In [28]:
print 'Median HH income in all NYC PUMAs:', nyc_pums['HINCP'].median()
print 'Median HH income in all NYC PUMAs, 2-person HH:', nyc_pums.query('NP == 2')['HINCP'].median()

Median HH income in all NYC PUMAs: 52400.0
Median HH income in all NYC PUMAs, 2-person HH: 60000.0


In [32]:
print 'Example median HH income in PUMA of 945 Aldus St, Bronx:', nyc_pums.query('NP == 2 & PUMA == 3710')['HINCP'].median()
print('Example median HH income in PUMA of 109-20 Queens Blvd, Queens:'), nyc_pums.query('NP == 2 & PUMA == 4108')['HINCP'].median()
print('Example median HH income in PUMA of 16 S Portland Ave, Brooklyn:'), nyc_pums.query('NP == 2 & PUMA == 4004')['HINCP'].median()

Example median HH income in PUMA of 945 Aldus St, Bronx: 22000.0
Example median HH income in PUMA of 109-20 Queens Blvd, Queens: 72150.0
Example median HH income in PUMA of 16 S Portland Ave, Brooklyn: 100000.0


In [11]:
# excluding group quarters and vacant household records
nyc_pums = nyc_pums.query('TYPE == 1 & BLD > 1 & BLD < 10')
nyc_pums = nyc_pums[nyc_pums['VACS'].isnull()]

In [12]:
print 'Percentage of households who own that are considered Low Income:', nyc_pums.query('TEN < 3')['AMI_80'].mean()
nyc_pums.query('TEN < 3').pivot_table(index='SizeGroup',columns='AgeGroup',values='AMI_80',aggfunc='mean')

Percentage of households who own that are considered Low Income: 0.364251928323


AgeGroup,Modern,PostWar,PreWar
SizeGroup,,,
Large,0.209896,0.360575,0.239879
Small,0.325652,0.382683,0.389609


In [13]:
print 'Percentage of households who rent that are considered Low Income:', nyc_pums.query('TEN == 3')['AMI_80'].mean()
nyc_pums.query('TEN == 3').pivot_table(index='SizeGroup',columns='AgeGroup',values='AMI_80',aggfunc='mean')

Percentage of households who rent that are considered Low Income: 0.646726031915


AgeGroup,Modern,PostWar,PreWar
SizeGroup,,,
Large,0.521380,0.672111,0.648297
Small,0.662688,0.635525,0.625901


In [14]:
print 'Also add length of residency category...'
print 'Percentage of households who rent that are considered Low Income:', nyc_pums.query('TEN == 3')['AMI_80'].mean()
nyc_pums.query('TEN == 3').pivot_table(index='SizeGroup',columns=['ResidencyGroup','AgeGroup'],values='AMI_80',aggfunc='mean')

Also add length of residency category...
Percentage of households who rent that are considered Low Income: 0.646726031915


ResidencyGroup   10Years                       20Years             30Years  \
AgeGroup          Modern   PostWar    PreWar   PostWar    PreWar   PostWar   
SizeGroup                                                                    
Large           0.791946  0.744360  0.720914  0.743861  0.740624  0.757032   
Small           0.756410  0.639512  0.664454  0.667737  0.666113  0.685328   

ResidencyGroup             Under10                      
AgeGroup          PreWar    Modern   PostWar    PreWar  
SizeGroup                                               
Large           0.732126  0.510909  0.605954  0.586149  
Small           0.697411  0.657860  0.629009  0.605733

In [15]:
# can we identify renters who live in condo buildings?
nyc_pums.query('TEN == 3 and FCONP == 1')
# it appears not

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP77,WGTP78,WGTP79,WGTP80,PUMA,AMI_60,AMI_80,AgeGroup,SizeGroup,ResidencyGroup


In [16]:
# dependents to test: HINCP, RNTP, GRNTP, NPF, TAXP
# independents to test: YBL (ord), TEN (c), BDSP, BLD, VACS - perhaps as a filter?
# also of interest: MV

In [17]:
# predict rent paid for renters
# mod = smf.ols(formula='RNTP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
# print(mod.summary())

In [18]:
# predict income for renters
# mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
# print(mod.summary())

In [19]:
# predict income for owners
# mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
# print(mod.summary())

In [20]:
# predict income for owners with property tax as independent, but without bedrooms or size of building (covairance)
# mod = smf.ols(formula='HINCP ~ TAXP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
# print(mod.summary())

In [21]:
# predict rent paid for renters with grouped building ages
# mod = smf.ols(formula='RNTP ~ BLD + BDSP + C(PUMA) + C(AgeGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
# print(mod.summary())

In [22]:
# predict income for renters with grouped building ages/sizes and length of residence
mod = smf.ols(formula='HINCP ~ NP + C(SizeGroup) + C(AgeGroup) + C(PUMA) + C(ResidencyGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  HINCP   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     264.2
Date:                Sun, 28 Feb 2016   Prob (F-statistic):               0.00
Time:                        19:51:49   Log-Likelihood:            -9.9401e+05
No. Observations:               78834   AIC:                         1.988e+06
Df Residuals:                   78771   BIC:                         1.989e+06
Df Model:                          62                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
Intercept       

In [23]:
logit_mod = smf.logit(formula='AMI_80 ~ C(SizeGroup) + C(AgeGroup) + C(PUMA) + C(ResidencyGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(logit_mod.summary())

         Current function value: 0.584122
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                 AMI_80   No. Observations:                78834
Model:                          Logit   Df Residuals:                    78772
Method:                           MLE   Df Model:                           61
Date:                Sun, 28 Feb 2016   Pseudo R-squ.:                  0.1006
Time:                        19:52:24   Log-Likelihood:                -46049.
converged:                      False   LL-Null:                       -51199.
                                        LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
Intercept                       27.1804   3.57e+05   7.61e-05      1.000        -7e+05     7e+05
C(SizeGroup

C:\Anaconda3\envs\py27\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [24]:
mfx = logit_mod.get_margeff()
print mfx.summary()

        Logit Marginal Effects       
Dep. Variable:                 AMI_80
Method:                          dydx
At:                           overall
                                  dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
C(SizeGroup)[T.Small]           -0.0435      0.004    -11.010      0.000        -0.051    -0.036
C(AgeGroup)[T.PostWar]           0.0419      0.006      6.502      0.000         0.029     0.055
C(AgeGroup)[T.PreWar]            0.0362      0.007      5.518      0.000         0.023     0.049
C(PUMA)[T.2602]                 -2.1726   7.12e+04  -3.05e-05      1.000      -1.4e+05   1.4e+05
C(PUMA)[T.3701]                 -5.3356   7.12e+04  -7.49e-05      1.000      -1.4e+05   1.4e+05
C(PUMA)[T.3702]                 -5.1751   7.12e+04  -7.27e-05      1.000      -1.4e+05   1.4e+05
C(PUMA)[T.3703]                 -5.3089   7.12e+04  -7.45e-05      1.000